<a href="https://colab.research.google.com/github/riccardopezzoni/Covid-19_Impact_on_Mobile_Networks/blob/main/IOT_WI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import csv
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics.cluster import completeness_score


ImportError: ignored

In [ ]:
!pip install sklearn.metrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement sklearn.metrics (from versions: none)
ERROR: No matching distribution found for sklearn.metrics


In [ ]:
##Used to load mount google drive, comment if not needed
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
##Edit to use different File path
file_path = '/content/drive/MyDrive/IOTWIProject/'
gt = pd.read_csv(file_path+'gt_csv.csv') # Grand Truth CSV
gt["Time"] = pd.to_datetime(gt["Time"])
gt = gt.set_index("Time")

capture = pd.read_csv(file_path+'capture2.csv') # Capture CSV
capture["Time"] = pd.to_datetime(capture["Time"])

capture = capture[capture["Time"] > pd.Timestamp(year=2022, month=8, day=26, hour = 13, minute =17)]
capture = capture[capture["Time"] < pd.Timestamp(year=2022, month=8, day=26, hour = 13, minute =37)]

capture = capture.set_index("Time")
capture = capture.drop(['No.', 'Source', 'Destination', 'Info'], axis=1)


# Data Formatting

In [ ]:
#Data Frame of TCP packets
tcp = capture[capture['Protocol'] == "TCP"]
tcp1 = tcp.resample('0.5S').median()
tcp1 = tcp1.rename(columns={"Length": "med_len_TCP"})
tcp2 = tcp.resample('0.5S').count().drop(['Length'], axis=1)
tcp2 = tcp2.rename(columns={"Protocol": "n_TCP"})
tcp1 = pd.concat([tcp1, tcp2], axis=1)
tcp1

,med_len_TCP,n_TCP
Time,,
2022-08-26 13:17:00.000,66.0,1
2022-08-26 13:17:00.500,66.0,2
2022-08-26 13:17:01.000,66.0,2
2022-08-26 13:17:01.500,66.0,3
2022-08-26 13:17:02.000,NaN,0
...,...,...
2022-08-26 13:36:57.000,66.0,3
2022-08-26 13:36:57.500,66.0,3
2022-08-26 13:36:58.000,66.0,2


In [ ]:
#Data Frame of UDP packets
udp = capture[capture['Protocol'] == "UDP"]
udp1 = udp.resample('0.5S').median()
udp1 = udp1.rename(columns={"Length": "med_len_UDP"})
udp2 = udp.resample('0.5S').count().drop(['Length'], axis=1)
udp2 = udp2.rename(columns={"Protocol": "n_UDP"})
udp1 = pd.concat([udp1, udp2], axis=1)
udp1

,med_len_UDP,n_UDP
Time,,
2022-08-26 13:17:00.000,1120.0,51
2022-08-26 13:17:00.500,1089.0,77
2022-08-26 13:17:01.000,1066.0,75
2022-08-26 13:17:01.500,1044.0,75
2022-08-26 13:17:02.000,1113.0,87
...,...,...
2022-08-26 13:36:57.500,1144.0,96
2022-08-26 13:36:58.000,1131.5,110
2022-08-26 13:36:58.500,1180.0,83


In [ ]:
#Data Frame of TLSv1.2 packets
tls = capture[capture['Protocol'] == "TLSv1.2"]
tls1 = tls.resample('0.5S').median()
tls1 = tls1.rename(columns={"Length": "med_len_TLS"})
tls2 = tls.resample('0.5S').count().drop(['Length'], axis=1)
tls2 = tls2.rename(columns={"Protocol": "n_TLS"})
tls1 = pd.concat([tls1, tls2], axis=1)
tls1

,med_len_TLS,n_TLS
Time,,
2022-08-26 13:17:00.000,116.0,1
2022-08-26 13:17:00.500,116.0,3
2022-08-26 13:17:01.000,116.0,2
2022-08-26 13:17:01.500,111.0,3
2022-08-26 13:17:02.000,NaN,0
...,...,...
2022-08-26 13:36:57.000,116.0,2
2022-08-26 13:36:57.500,113.5,4
2022-08-26 13:36:58.000,116.0,2


In [ ]:
#Data Frame of WireGuard packets
wg = capture[capture['Protocol'] == "WireGuard"]
wg1 = wg.resample('0.5S').median()
wg1 = wg1.rename(columns={"Length": "med_len_WG"})
wg2 = wg.resample('0.5S').count().drop(['Length'], axis=1)
wg2 = wg2.rename(columns={"Protocol": "n_WG"})
wg1 = pd.concat([wg1, wg2], axis=1)
wg1

,med_len_WG,n_WG
Time,,
2022-08-26 13:17:01.500,123.0,1
2022-08-26 13:17:02.000,123.0,2
2022-08-26 13:17:02.500,NaN,0
2022-08-26 13:17:03.000,NaN,0
2022-08-26 13:17:03.500,NaN,0
...,...,...
2022-08-26 13:29:11.000,NaN,0
2022-08-26 13:29:11.500,NaN,0
2022-08-26 13:29:12.000,NaN,0


In [ ]:
gt = gt.resample('0.5S').median().ffill()
gt = gt.query("Time < Timestamp('2022-08-26 13:37:00')")
gtlables = gt.to_numpy()


In [ ]:
#Creation of the merged DataFrame that will be provided as imput to the classifications algorithms
merged = pd.concat([tcp1, udp1, tls1, wg1], axis=1)
merged = merged.fillna(0)
merged

,med_len_TCP,n_TCP,med_len_UDP,n_UDP,med_len_TLS,n_TLS,med_len_WG,n_WG
Time,,,,,,,,
2022-08-26 13:17:00.000,66.0,1.0,1120.0,51,116.0,1.0,0.0,0.0
2022-08-26 13:17:00.500,66.0,2.0,1089.0,77,116.0,3.0,0.0,0.0
2022-08-26 13:17:01.000,66.0,2.0,1066.0,75,116.0,2.0,0.0,0.0
2022-08-26 13:17:01.500,66.0,3.0,1044.0,75,111.0,3.0,123.0,1.0
2022-08-26 13:17:02.000,0.0,0.0,1113.0,87,0.0,0.0,123.0,2.0
...,...,...,...,...,...,...,...,...
2022-08-26 13:36:57.500,66.0,3.0,1144.0,96,113.5,4.0,0.0,0.0
2022-08-26 13:36:58.000,66.0,2.0,1131.5,110,116.0,2.0,0.0,0.0
2022-08-26 13:36:58.500,66.0,2.0,1180.0,83,116.0,2.0,0.0,0.0


In [ ]:
#Split in train set and test set
X_train, X_test, y_train, y_test = train_test_split(merged, gt, train_size=0.25, random_state=15)


# Logistic Regression Algorithm


In [ ]:
logreg = LogisticRegression(random_state=3)
logreg.fit(X_train, y_train)
y_pred_logreg = logreg.predict(X_test)
y_pred_prob_logreg = logreg.predict_proba(X_test)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
confusion_matrix(logreg, X_test, y_test)  
plt.show()

TypeError: ignored

In [ ]:
target_names = ['Not Present', 'Present']
print(classification_report(y_test, y_pred_logreg, target_names=target_names))


# Random Forest Algorithm


In [ ]:
#Random Forest Algorithm
ranforest = RandomForestClassifier(n_estimators= 200, random_state=3)
ranforest.fit(X_train, y_train)
y_pred_ranforest = ranforest.predict(X_test)

In [ ]:
plot_confusion_matrix(ranforest, X_test, y_test)  


In [ ]:
target_names = ['Not Present', 'Present']
print(classification_report(y_test, y_pred_ranforest, target_names=target_names))

# K-Nearest Neighbors Algorithm

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

In [ ]:
plot_confusion_matrix(knn, X_test, y_test)  


In [ ]:
target_names = ['Not Present', 'Present']
print(classification_report(y_test, y_pred_knn, target_names=target_names))

#Analisys of Mistakes

In [ ]:
results_logreg = pd.DataFrame([X_test.index,y_pred_logreg]).T
results_logreg = results_logreg.set_index(0)
results_logreg = results_logreg.rename(columns={1: "logreg"})
results_logreg = results_logreg.sort_values(0)
results_ranforest = pd.DataFrame([X_test.index,y_pred_ranforest]).T
results_ranforest = results_ranforest.set_index(0)
results_ranforest = results_ranforest.rename(columns={1: "ranforest"})
results_ranforest = results_ranforest.sort_values(0)
results_knn = pd.DataFrame([X_test.index,y_pred_knn]).T
results_knn = results_knn.set_index(0)
results_knn = results_knn.rename(columns={1: "knn"})
results_knn = results_knn.sort_values(0)
results = pd.concat([results_logreg, results_ranforest, results_knn, y_test], axis=1)
results = results.rename(columns={"Person": "Grand_Truth"})

results.query('logreg != Grand_Truth | knn != Grand_Truth | ranforest!= Grand_Truth')


In [ ]:
udp1.plot(figsize=(50, 25))

In [ ]:
results.plot(figsize=(50, 25))

#Export for NodeRed

In [ ]:
logreg_total = results_logreg.join(y_train, how='outer')
logreg_total = logreg_total.fillna(0)
logreg_total['Logreg'] = logreg_total['logreg'] + logreg_total['Person']
logreg_total = logreg_total.drop(['logreg', 'Person'], axis=1)
logreg_total.index.name = 'Date'

ranforest_total = results_ranforest.join(y_train, how='outer')
ranforest_total = ranforest_total.fillna(0)
ranforest_total['Ranforest'] = ranforest_total['ranforest'] + ranforest_total['Person']
ranforest_total = ranforest_total.drop(['ranforest', 'Person'], axis=1)
ranforest_total.index.name = 'Date'

knn_total = results_knn.join(y_train, how='outer')
knn_total = knn_total.fillna(0)
knn_total['Knn'] = knn_total['knn'] + knn_total['Person']
knn_total = knn_total.drop(['knn', 'Person'], axis=1)
knn_total.index.name = 'Date'

In [ ]:
logreg_total.to_csv('logreg_total.csv', quoting=csv.QUOTE_NONNUMERIC)
ranforest_total.to_csv('ranforest_total.csv', quoting=csv.QUOTE_NONNUMERIC)
knn_total.to_csv('knn_total.csv', quoting=csv.QUOTE_NONNUMERIC)
